# EmergencyZIP AI

## Intro

As a Data Scientist in Healthcare with 2+ years of industry and work experience, one of the valuable pieces of knowledge that I learned on the job is that Zipcode (Postal Code) is indeed an important factor in determining an individual's health status (thus, wherever the individual lives can determine how healthy or unhealthy an individual can be). Also, healthcare, especially the quality of healthcare, is also an important factor in determining the standard of life of an area as well. However, many people may not be familiar with all the medical facilities that their area may provide or may not know the best one to go to based on what illness or medical condition that they may experience now. Thus, I created the EmergencyZIP AI to help individuals determine the best medical facilities to attend based on the symptoms or conditions that they may be experiencing right now, and to take greater consideration of zipcode/postal code as one of the most important factors of an individual's health status. 

## Process

The AI utilizes Structured Output/JSON Mode/Controlled Generation, Few-Shot Prompting, and Grounding to operate. First, it uses JSON Mode to process patient demographic information and structures it to a formalized JSON form (just like filling out a patient form to receive medical care except simplier and more abstract). Then it utilizes Few-Shot Prompting and Structured Output/Controlled Generation to create a request for the AI. Then it utilizes grounding to find the best and most accurate diagnosis for the patient based on the current symptoms/conditions that the patient is facing now. Then using the diagnosis and the current zipcode of the patient (the AI uses Few-Shot Prompting to extract the zipcode of the patient from the JSON form), the AI finds the nearest and best medical facilities that can treat the patient. The AI also uses grounding to inform the patient payment and insurance plan options that the suggested medical facilities can accept as payment for the treatment. 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/patient-symptom-dataset/patient_symptom_data.csv


In [2]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.3 MB/s eta 0:00:00


In [3]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

In [4]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [5]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [6]:
client = genai.Client(api_key=GOOGLE_API_KEY)

## Generative AI Capabilities Used and Reasoning 

These are the 3 Generative AI Capabilities Used in this AI and why they are used in this AI: 
* **Structured Output/JSON Mode/Controlled Generation**: For JSON Mode, I want to somewhat mimic the forms that patients fill out when they are requesting care from a medical facility, but simplier and more abstract. For Structured Output and Controlled Generation, I want my responses to be as accurate and precise as possible where creativity may not be required but accuracy is definitely required to get the best responses as possible.
* **Few-Shot Prompting**: First of all, I want to create requests for the AI to know who it is providing information for, but I want the formatting of the requests to be as consistent as possible so that the information does not get muddled up which would make things more difficult for the AI to process and provide information for. Then, I also want to extract the zipcode from the JSON form, but sometimes the AI can hallucinate and return irrelevant or incorrect information, so I want the let the AI know how to return the zipcode correctly.
* **Grounding**: This is the central capability of the AI since the AI is responsible for finding all the relevant possible nearest locations to the patient based on postal code/zipcode. Since we are exploring so many zipcodes and it is possible that the AI does not know every zipcode or medical facilities per zipcode, we need to rely on external sources to get the best answers possible. However, since we don't have a document listing out every medical facility per zipcode or all zipcodes, we would need to rely on the internet to give us answers. 

### Generative AI Capability #1: Structured Output/JSON Mode/Controlled Generation

#### JSON Mode

In [7]:
import typing_extensions as typing

class MedicalPatient(typing.TypedDict): 
    zipcode: int
    age: int
    gender: str
    symptoms: list[str]

def get_json_form(patient_symptom_entry): 
    entry_str = patient_symptom_entry.to_string()
    json_response = client.models.generate_content(
        model='gemini-2.0-flash', 
        config=types.GenerateContentConfig(
            temperature=0.1, 
            response_mime_type="application/json", 
            response_schema=MedicalPatient
        ), 
        contents=entry_str
    )
    json_form = json_response.text
    return json_form

#### Structured Output/Controlled Generation

In [8]:
def get_general_model_config(temperature, top_p, max_output_tokens): 
    return types.GenerateContentConfig(
        temperature=temperature, 
        top_p=top_p, 
        max_output_tokens=max_output_tokens
    )

### Generative AI Capability #2: Few-Shot Prompting

In [9]:
def few_shot_func(model_input, prompt, config): 
    few_shot_response = client.models.generate_content(
        model='gemini-2.0-flash', 
        config=config,
        contents=[prompt, model_input]
    )
    str_response = few_shot_response.text
    return str_response

### Generative AI Capability #3: Grounding

In [10]:
def grounding_func(input_prompt): 
    config_with_search = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
    )
    response = client.models.generate_content(
        model='gemini-2.0-flash', 
        contents=input_prompt, 
        config=config_with_search,
    )
    rc = response.candidates[0]
    return rc.content.parts[0].text

## EmergencyZIP AI Test Run

Kaggle Notebooks seem to not work or do well with Python input, so instead, we generated a dataset of patients with relevant demographic information and their symptoms through ChatGPT (so the dataset below is ChatGPT-generated), and then the code picks a random patient to test run the AI to make sure it actually works as intended. 

In [11]:
patient_symptom_df = pd.read_csv('/kaggle/input/patient-symptom-dataset/patient_symptom_data.csv')
patient_symptom_df

,PatientID,ZipCode,Age,Gender,Symptoms
0,P001,70112,32,Female,Joint pain;Runny nose;Loss of taste;Sore throat
1,P002,15213,87,Female,Cough;Fever;Fatigue;Chest pain;Joint pain
2,P003,46201,21,Non-binary,Swelling;Blurred vision;Rash
3,P004,20001,46,Male,Chest tightness;Sneezing;Fever;Breathing diffi...
4,P005,96813,38,Non-binary,Vomiting;Headache;Runny nose;Chest pain;Breath...
...,...,...,...,...,...
95,P096,92101,71,Non-binary,Low-grade fever;Joint pain;Headache
96,P097,20001,80,Female,Vomiting;Abdominal cramps;Chest tightness;Abdo...
97,P098,85001,77,Female,Loss of smell;Abdominal pain
98,P099,84101,90,Non-binary,Fatigue;Blurred vision


In [12]:
patient_index = random.randint(0, 99)
patient_json_form = get_json_form(patient_symptom_df.iloc[patient_index])
print(patient_json_form)

{
"zipcode": 64106,
"age": 31,
"gender": "Male",
"symptoms": ["Sneezing", "Low-grade fever"]
}


In [13]:
model_config = get_general_model_config(0.1, 1, 500)

few_shot_prompt = """Parse JSON into a string request and only return output: 

EXAMPLE: 
{
"zipcode": 78664, 
"age": 30, 
"gender": "Female", 
"symptoms": ["Cough", "Fever", "Sore throat"]
}
"30-year old female patient from zipcode 78664 currently experiencing symptoms of cough, fever, and sore throat."

EXAMPLE: 
{
"zipcode": 77055, 
"age": 25, 
"gender": "Non-binary", 
"symptoms": ["Joint pain", "Loss of smell"]
}
"25-year old non-binary patient from zipcode 77055 currently experiencing symptoms of joint pain and loss of smell."

EXAMPLE: 
{
"zipcode": 78681, 
"age": 15, 
"gender" "Male", 
"symptoms": ["Fatigue", "Loss of Vision", "Headache", "Joint pain"]
}
"15-year old male patient from zipcode 78681 currently experiencing symptoms of fatigue, loss of vision, headache, and joint pain."
"""

patient_medical_request = few_shot_func(patient_json_form, few_shot_prompt, model_config)
print(patient_medical_request)

"31-year old male patient from zipcode 64106 currently experiencing symptoms of sneezing and low-grade fever."



In [14]:
diagnosis_prompt = patient_medical_request + "What disease are they most likely to experience now? Only return the singular most likely disease, please."
diagnosis = grounding_func(diagnosis_prompt)
Markdown(diagnosis)

Given the symptoms of sneezing and a low-grade fever, the most likely disease the 31-year-old male patient is experiencing is the **common cold**.


In [15]:
model_config_two = get_general_model_config(0, 1, 50)

few_shot_prompt_two = """Parse JSON to return only the Zipcode: 

EXAMPLE: 
{
"zipcode": 78664, 
"age": 30, 
"gender": "Female", 
"symptoms": ["Cough", "Fever", "Sore throat"]
}
78664

EXAMPLE: 
{
"zipcode": 77055, 
"age": 25, 
"gender": "Non-binary", 
"symptoms": ["Joint pain", "Loss of smell"]
}
77055

EXAMPLE: 
{
"zipcode": 78681, 
"age": 15, 
"gender" "Male", 
"symptoms": ["Fatigue", "Loss of Vision", "Headache", "Joint pain"]
}
78681
"""

patient_zipcode = few_shot_func(patient_json_form, few_shot_prompt_two, model_config_two)
print(patient_zipcode)

64106



In [16]:
recommendations_prompt = diagnosis + 'Can you get me the nearest best-rated medical facilities with their addresses that can treat the disease for zip code ' + str(patient_zipcode) + '?'
recommendations = grounding_func(recommendations_prompt)
Markdown(recommendations)

Okay, I can help you find some medical facilities near the 64106 zip code that can treat a common cold. Since you're looking for the "nearest best-rated," I'll provide a few options from the search results, including urgent care centers and primary care physicians, as these are typically the most appropriate for cold symptoms. Note that "best-rated" is subjective, so I'm including facilities that are well-known and have good reputations based on the search results.

Here are some options:

**Urgent Care Centers:**

*   **T-Mobile Center Urgent Care (The University of Kansas Health System):** Located at 1403 Grand Blvd, Kansas City, MO 64106. This is within the 64106 zip code. It's part of a larger health system, which could be beneficial. Hours are Mon-Fri 8:30 AM - 5 PM.
*   **Concentra Crossroads:** Located at 200 Southwest Boulevard, Kansas City, MO 64108. This is very close to the 64106 zip code. They offer urgent care services and treat colds and flu. Hours are Mon-Fri, 8 AM - 5 PM.
*   **Saint Luke's Urgent Care:** While not directly in the 64106 zip code, they have a location at 8880 NE 82nd Ter Kansas City, MO 64158 which may be convenient. They treat fever, flu, cough, and sore throats. Hours are Mon-Fri: 7 a.m. – 7 p.m.; Saturday and Sunday: 8 a.m. – 5 p.m.
*   **NextCare Urgent Care: Ambassador:** Located at 10015 N Ambassador Dr Ste. 100, Kansas City, MO 64153. Hours are Mon-Fri 8:00 AM - 8:00 PM, Sat 8:00 AM - 6:00 PM, and Sun 8:00 AM - 6:00 PM.

**Primary Care Physicians:**

*   **University Health Primary Care:** They have multiple locations.
    *   John W. Bluford Medical Pavilion (downtown Kansas City): They have family physicians who treat patients of all ages. Call 816-404-3855 to make an appointment.
    *   2211 Charlotte Street Kansas City, MO 64108, call 816.404.7030
*   **Ascension Medical Group:** Jessica L Brunkhorst, MD (Pediatrics), located at 2401 Gillham Rd Kansas City, MO 64108.

**Important Considerations:**

*   **Check Hours and Availability:** It's always a good idea to call before you go, to confirm their hours of operation and ensure they are accepting new patients or walk-ins.
*   **Insurance:** Verify that the facility accepts your insurance.
*   **Severity of Symptoms:** Since you mentioned the patient has a low-grade fever, it's likely a common cold. However, if symptoms worsen or are accompanied by difficulty breathing or other serious concerns, seek emergency care.
*   **Virtual Care:** Some facilities offer virtual visits, which could be a convenient option.

I hope this helps you find the best medical care for the patient!


In [17]:
payments_prompt = "For each of the locations recommended to me: " + recommendations + " what types of payments and insurance plans are accepted for patients coming in for treatment?"
payments = grounding_func(payments_prompt)
Markdown(payments)

Okay, I can research the payment types and insurance plans accepted at the medical facilities I previously recommended. Since I don't have real-time access to each facility's specific accepted plans, I'll provide general information and steps you can take to verify the specifics:



## Possible Next Steps